<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/zoomapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import requests
import json
import pandas as pd
import google.auth
import streamlit as st
from google.cloud import bigquery
from google.oauth2.credentials import Credentials
from flask import Flask
from pyngrok import ngrok

In [ ]:
# This is a streamlit solution for the next 5 blocks
!pip install jedi
!pip install streamlit -q



In [ ]:
%%writefile app.py
st.write('#D.cypher')
st.write('##Insight.IQ')

Writing app.py


In [ ]:
!pip install pyngrok
# !pip install flask==0.12.2

In [ ]:
ngrok_auth_token = ''
port_number = '5000'
# ngrok.set_auth_token(ngrok_auth_token)
# public_url = ngrok.connect(5000).public_url

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_auth_token)

In [ ]:
!nohup streamlit run app.py --server.port port_number &
public_url =  ngrok.connect(port_number).public_url
print(public_url)

In [ ]:
  # Get the authorization code from the query parameters
  # authorization_code = request.args.get("code")

  # Exchange the authorization code for an access token
  response = requests.post("https://zoom.us/oauth/token", data={
      "grant_type": "authorization_code",
      "redirect_uri": public_url,
      "client_id": CLIENT_ID,
      "client_secret": CLIENT_SECRET
  })

  # Check the response status code
  if response.status_code != 200:
      raise Exception("Failed to get access token: " + response.text)

  # Extract the access token from the response
  access_token = response.json()["access_token"]

  # Store the access token for later use
  # ...

  return redirect("/")

In [ ]:
# This is a flask solution to run app for auth token
# app = Flask(__name__)
# ngrok.set_auth_token(ngrok_auth_token)
# public_url =  ngrok.connect(port_number).public_url

# @app.route("/")
# def home():
#     return f"Running Flask on Google Colab!"

# print(f"To access the Global link please click {public_url}")

# app.run(port=port_number)

In [ ]:
CLIENT_ID = ''
CLIENT_SECRET = ''
ACCOUNT_ID = ''
redirect_uri = 'http://.ngrok.io/'

In [ ]:
from flask import Flask, request, redirect
import requests

app = Flask(__name__)

@app.route("/oauth/redirect")
def handle_redirect():
    # Get the authorization code from the query parameters
    authorization_code = request.args.get("code")

    # Exchange the authorization code for an access token
    response = requests.post("https://zoom.us/oauth/token", data={
        "grant_type": "authorization_code",
        "redirect_uri": public_url,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    })

    # Check the response status code
    if response.status_code != 200:
        raise Exception("Failed to get access token: " + response.text)

    # Extract the access token from the response
    access_token = response.json()["access_token"]

    # Store the access token for later use
    # ...

    return redirect("/")




In [ ]:
# # Set Google OAuth credentials
# creds = Credentials.from_authorized_user_info(info={"access_token": '$ACCESS_TOKEN',
#                                                      "refresh_token": '$REFRESH_TOKEN',
#                                                      "client_id": '$CLIENT_ID',
#                                                      "client_secret": '$CLIENT_SECRET'})
# print(creds.refresh)

In [ ]:
# Refresh the access token
# creds.refresh(google.auth.transport.requests.Request())
# response = requests.get("https://zoom.us/oauth/token?response_type=code&client_id={{CLIENT_ID}}&redirect_uri={{redirectUriOAuthDev}}")
# Make API call to Zoom to get transcriptions
response = requests.get("https://api.zoom.us/v2/past_webinars/transcripts", headers={"Authorization": "Bearer " + access_token})

# Check response status
if response.status_code != 200:
    raise Exception("Failed to get transcriptions: " + response.text)

# Extract transcriptions from response
transcriptions = response.json()


In [ ]:
# # After joining a session, call client.getLiveTranscriptionClient() to get the live transcription and translation client.
# const liveTranscriptionTranslation = client.getLiveTranscriptionClient();
# # To start live transcription and translation, call the liveTranscriptionTranslation.startLiveTranscription() function.
# liveTranscriptionTranslation.startLiveTranscription();
# # To receive speech text, add the following event listener.
# client.on(`caption-message`, (payload) => {
#   console.log(payload)
#   console.log(`${payload.displayName} said: ${payload.text}`);
# });

In [ ]:
# Create a pandas dataframe from the transcriptions
df = pd.DataFrame(transcriptions)


In [ ]:
# Use Google Cloud Natural Language API to extract classifications
def extract_classifications(text):
    # Request Body
    body = {
        "document": {
            "type": "PLAIN_TEXT",
            "content": text
        },
        "encoding_type": "UTF8"
    }

    # Make API call to Google Cloud Natural Language API
    response = requests.post("https://language.googleapis.com/v1/documents:analyzeEntities",
                             headers={"Authorization": "Bearer " + credentials.token,
                                      "Content-Type": "application/json"},
                             json=body)

    # Check response status
    if response.status_code != 200:
        raise Exception("Failed to extract classifications: " + response.text)

    # Extract entities from response
    entities = response.json()["entities"]

    # Return entity types
    return [entity["type"] for entity in entities]


In [ ]:
# Add classifications to the dataframe
df["classifications"] = df["transcription"].apply(extract_classifications)

# Load dataframe into BigQuery
bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
table_id = PROJECT_ID + "." + DATASET_ID + "." + TABLE_ID
df.to_gbq(table_id, if_exists="replace")

# Print success message
print("Data loaded successfully to BigQuery")